In [26]:
# from the provided file 'US_Presidents_unordered.txt':
    # seperate the data into the following in order:
        # first_name
        # middle_intial
        # last_name
        # begin_term
        # end_term
        # presedential order

    # Create a report as a new txt file containing the following in presidential order:
        # order_number
        # first_name 
        # middle_initial
        # last_name
        # begin_term
        # end_term 
        # years_in_office

    # Save the data into a database into the respective columns in presidential order:
        # order text,  
        # first_name text, 
        # middle_initial text,
        # last_name text 
        # begin_term text, 
        # end_term text, 
        # years_in_office real,

In [18]:
import sqlite3

data = {}
with open("US_Presidents_unordered.txt", "r") as file:
    line = file.readline()
    while line:
        # Skip first line in file
        if 'Name,Term_Start,Term_End' not in line: 
            # Filter / , ' . characters out of the string
            line = line.replace('\'','').replace('.','').replace(',',' ')

            # Change the string into a list of items seperated by spaces
            line = line.split()

            # If there is no middle name
            if len(line) == 5:

                # Insert n/a as the middle name
                line.insert(2, 'n/a')

            # Store presidents years in office at the end of the list
            # (Using the difference of the end/start dates)
            line.append(str(int(line[5][-4:]) - int(line[4][-4:])))

            # store data into a dictionary with the presidents order number as a key
            data[int(line[0])] = line
        line = file.readline()

# Sort dictionary by it's items index, then convert back to a dictionary
data = dict(sorted(data.items()))

In [20]:
# Open the file in write mode (will be created if it doesn't exist)
file = open("solution.txt", 'w')
# Format the list to a writeable string
[file.write(' '.join(data[line])+'\n') for line in data]
file.close()

In [24]:
# Save that data to the database
connection = sqlite3.connect('united_states.db')
cursor = connection.cursor()
cursor.execute('''create table if not exists presidents (
        order_number text, 
        first_name text, 
        middle_initial text, 
        last_name text, 
        begin_term text, 
        end_term text, 
        years_in_office real
)''')
[cursor.execute('insert into presidents values(?,?,?,?,?,?,?)', data[item]) for item in data]
connection.commit()
connection.close()